# Parquet file

### Introduction to `parquet` file 

Parquet is an open source column oriented data store created for the Hadoop ecosystem. Wikipedia has a very good entry on column oriented database. I’ll summarise the content here so that the continuity is maintained. Please note that I've shamelessly picked the example from Wikipedia. 

In a database management system (DBMS), commonly and intuitively, data tables are stored as a row rather than columns. Unlike this approach, in the column oriented data storage, data tables are stored as columns. The following example will clear out the difference. A database might have a table as shown below.

| RowID | EmpID   | LastName | FirstName   | Salary |
|------|------|------|------|------|
| 001  | 10| Smith| Joe| 40000| 
|------|------|------|------|------|
| 002  | 12| Jones | Mary   |50000 |  
|------|------|------|------|------|
| 003  | 11|Johnson|Cathy|44000|
|------|------|------|------|------|
| 004  |22|Jones|Bob|55000|


The 2D tabular representation of information is actually an abstraction of the manner in which data is stored in a hardware.  Storage hardware require the data to be serialized in some form. Serialization is done in such a manner that the number of seeks in the hard disk can be minimized. This is because ‘seek’ is the most expensive operation in hard disks. 

There are two common ways to serialize the data. One is the `row oriented` wherein all the values of first row are serialized together, then the second row and so on and so forth. Following this approach, the table above shall be serialized in the following manner:

```
001:10,Smith,Joe,40000;
002:12,Jones,Mary,50000;
003:11,Johnson,Cathy,44000;
004:22,Jones,Bob,55000;
```

Unlike this approach, in `column oriented` approach, values of a column are serialized at a time. Using this type of serialization, data shall be stored in the following manner.

```
10:001,12:002,11:003,22:004;
Smith:001,Jones:002,Johnson:003,Jones:004;
Joe:001,Mary:002,Cathy:003,Bob:004;
40000:001,50000:002,44000:003,55000:004;
```

Two important benefits of a column oriented data store are: One, it reduces the amount of time needed to read the data from the disk. Most of the time, we are concerned with applying a filter on a column and retrieving the data from other columns. In such scenarios, reading from a row oriented storage would mean reading all the chunks of data, regardless of whether the column is of interest. Two, the column data offers the benefit of storage size optimization. Many popular data compression schemes, makes use of the similarity of adjacent data to compress.  

### Reading a parquet file

The parquet file for this notebook has been taken from __[this repo](https://github.com/jcrobak/parquet-python/tree/master/test-data)__. 

In [13]:
import sys
import os

LINE_LENGTH = 200

In [14]:
sqlContext = SQLContext(sparkContext=sc)

Load the file from the local file system into a SQL DataFrame. 

In [23]:
parquetFile = sqlContext.read.parquet('./data/nation.plain.parquet')
type(parquetFile)

pyspark.sql.dataframe.DataFrame

Store the DataFrame into an "in-memory temporary table".

In [24]:
parquetFile.registerTempTable("parquetFile")

In [25]:
nations_all_sql = sqlContext.sql("SELECT * FROM parquetFile")
type(nations_all_sql)

pyspark.sql.dataframe.DataFrame

Note that in the following line, we will first convert the dataframe into an RDD and then use a `map`. Prior to Spark 2.0, spark_df.map would alias to spark_df.rdd.map(). With Spark 2.0, you must explicitly call .rdd first. This tip has been taken from __[here](https://stackoverflow.com/questions/39535447/attributeerror-dataframe-object-has-no-attribute-map)__.

In [26]:
nations_all = nations_all_sql.rdd.map(lambda p: "Country: {0:15} Ipsum Comment: {1}".format(p.name, p.comment_col))

In [27]:
print("All Nations and Comments -- `SELECT * FROM parquetFile`")
print "========================================================="
for nation in nations_all.collect():
    print(nation)

All Nations and Comments -- `SELECT * FROM parquetFile`
Country: ALGERIA         Ipsum Comment:  haggle. carefully final deposits detect slyly agai
Country: ARGENTINA       Ipsum Comment: al foxes promise slyly according to the regular accounts. bold requests alon
Country: BRAZIL          Ipsum Comment: y alongside of the pending deposits. carefully special packages are about the ironic forges. slyly special 
Country: CANADA          Ipsum Comment: eas hang ironic, silent packages. slyly regular packages are furiously over the tithes. fluffily bold
Country: EGYPT           Ipsum Comment: y above the carefully unusual theodolites. final dugouts are quickly across the furiously regular d
Country: ETHIOPIA        Ipsum Comment: ven packages wake quickly. regu
Country: FRANCE          Ipsum Comment: refully final requests. regular, ironi
Country: GERMANY         Ipsum Comment: l platelets. regular accounts x-ray: unusual, regular acco
Country: INDIA           Ipsum Comment: ss excuses cajo

In [28]:
nations_filtered_sql = sqlContext.sql("SELECT name FROM parquetFile WHERE name LIKE '%UNITED%'")

In [21]:
nations_filtered = nations_filtered_sql.rdd.map(lambda p: "Country: {0:20}".format(p.name))

In [22]:
print "=============================================================================="
print("Nations Filtered -- `SELECT name FROM parquetFile WHERE name LIKE '%UNITED%'`")
print "=============================================================================="
for nation in nations_filtered.collect():
    print(nation)

Nations Filtered -- `SELECT name FROM parquetFile WHERE name LIKE '%UNITED%'`
Country: UNITED KINGDOM      
Country: UNITED STATES       
